<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/temp_hosting/rel_strength.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import yfinance as yf
# Example stock data (replace with your actual data)
nifty50_url = "https://raw.githubusercontent.com/anirbanghoshsbi/data/main/ind_nifty50list.csv"
nifty50 = pd.read_csv(nifty50_url)
tickers = nifty50['Symbol'].apply(lambda x: x + ".NS").tolist()
# Bank Nifty Stocks
nifty_stocks = tickers
start_date = '2023-01-02'

def download_data(stock_list,start_date):
    data = yf.download(stock_list, start=start_date)
    data = data[['Adj Close']]
    return data

# Example benchmark index data (replace with your actual data)
benchmark_index = {
    'Index': ['MyIndex'],
    'Value': [2000]
}

# Create DataFrames
stock_df = download_data(nifty_stocks, start_date)
benchmark_df = yf.download('^nsei',start_date)

# Assuming stock_data is your DataFrame with MultiIndex columns

# Reset the index to flatten it
stock_df.reset_index(inplace=True)

# Reset the columns index to flatten the MultiIndex
stock_df.columns = stock_df.columns.get_level_values(1)

# Convert index to datetime
stock_df.columns.values[0]='Date'

# Calculate percentage change for each stock
percentage_change_data = stock_df.set_index('Date').pct_change(90)*100

# Create a new DataFrame to store the percentage change data
percentage_change_df = pd.DataFrame(percentage_change_data)





benchmark_df['Percentage Change'] = benchmark_df['Adj Close'].pct_change(90) * 100

# Remove NaN values
stock_df.dropna(inplace=True)
benchmark_df.dropna(inplace=True)



[*********************100%%**********************]  50 of 50 completed
[*********************100%%**********************]  1 of 1 completed


In [2]:
merged_df = percentage_change_df.join(benchmark_df['Percentage Change'], how='inner')

In [3]:
covariance = merged_df.cov()['Percentage Change']
benchmark_variance = np.var(benchmark_df['Percentage Change'])
beta = covariance / benchmark_variance

In [23]:
# Get the last row of stock_df
last_row = percentage_change_df.iloc[-1]  # or stock_df.tail(1)

# Create a new DataFrame with the last row and the beta series
new_df = pd.DataFrame({
    'Percentage_change': last_row,
    'Beta': beta,
    'volatility':np.std(percentage_change_df),


})



In [24]:
new_df

,Percentage_change,Beta,volatility
ADANIENT.NS,7.794867,4.317285,24.874266
ADANIPORTS.NS,27.127025,3.353750,21.760110
APOLLOHOSP.NS,15.385955,1.395801,8.545330
ASIANPAINT.NS,-11.802468,0.424812,11.427237
AXISBANK.NS,-0.061888,0.598768,6.195021
BAJAJ-AUTO.NS,43.492366,2.346119,17.728739
BAJAJFINSV.NS,-6.409845,0.499463,7.491205
BAJFINANCE.NS,-7.975195,-0.195733,13.473578
BHARTIARTL.NS,33.232856,1.135025,7.570602
BPCL.NS,36.174736,3.984977,29.606387


In [25]:
# Reset the index of the new DataFrame
new_df = new_df.dropna().reset_index()


In [26]:
new_df['Relative_Strength']=new_df['Percentage_change']/benchmark_df['Percentage Change'].iloc[-1]

In [27]:
# Optional: Clustering
X = new_df[['Relative_Strength', 'volatility','Beta','Percentage_change']].values
kmeans = KMeans(n_clusters=10)
kmeans.fit(X)
new_df['Cluster'] = kmeans.labels_

# Print clustering results
print(new_df[['index', 'Cluster']])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


            index  Cluster
0     ADANIENT.NS        9
1   ADANIPORTS.NS        5
2   APOLLOHOSP.NS        3
3   ASIANPAINT.NS        2
4     AXISBANK.NS        0
5   BAJAJ-AUTO.NS        6
6   BAJAJFINSV.NS        4
7   BAJFINANCE.NS        4
8   BHARTIARTL.NS        1
9         BPCL.NS        5
10   BRITANNIA.NS        4
11       CIPLA.NS        7
12   COALINDIA.NS        5
13    DIVISLAB.NS        8
14     DRREDDY.NS        3
15   EICHERMOT.NS        3
16      GRASIM.NS        3
17     HCLTECH.NS        8
18    HDFCBANK.NS        4
19    HDFCLIFE.NS        2
20  HEROMOTOCO.NS        7
21    HINDALCO.NS        7
22  HINDUNILVR.NS        2
23   ICICIBANK.NS        8
24  INDUSINDBK.NS        4
25        INFY.NS        4
26         ITC.NS        4
27    JSWSTEEL.NS        0
28   KOTAKBANK.NS        2
29          LT.NS        8
30        LTIM.NS        2
31         M&M.NS        7
32      MARUTI.NS        7
33   NESTLEIND.NS        0
34        NTPC.NS        1
35        ONGC.NS        6
3

In [28]:
new_df

,index,Percentage_change,Beta,volatility,Relative_Strength,Cluster
0,ADANIENT.NS,7.794867,4.317285,24.874266,1.076692,9
1,ADANIPORTS.NS,27.127025,3.353750,21.760110,3.747011,5
2,APOLLOHOSP.NS,15.385955,1.395801,8.545330,2.125237,3
3,ASIANPAINT.NS,-11.802468,0.424812,11.427237,-1.630255,2
4,AXISBANK.NS,-0.061888,0.598768,6.195021,-0.008548,0
5,BAJAJ-AUTO.NS,43.492366,2.346119,17.728739,6.007529,6
6,BAJAJFINSV.NS,-6.409845,0.499463,7.491205,-0.885381,4
7,BAJFINANCE.NS,-7.975195,-0.195733,13.473578,-1.101601,4
8,BHARTIARTL.NS,33.232856,1.135025,7.570602,4.590400,1
9,BPCL.NS,36.174736,3.984977,29.606387,4.996757,5


In [29]:
df_cluster_3 = new_df[new_df['Cluster'] == 0]

In [30]:
df_cluster_3

,index,Percentage_change,Beta,volatility,Relative_Strength,Cluster
4,AXISBANK.NS,-0.061888,0.598768,6.195021,-0.008548,0
27,JSWSTEEL.NS,3.911697,1.238498,7.375658,0.540316,0
33,NESTLEIND.NS,-0.198713,0.422474,6.669678,-0.027448,0
44,TCS.NS,4.546637,1.029892,7.095764,0.628020,0
45,TECHM.NS,4.289848,0.295035,6.745294,0.592550,0
46,TITAN.NS,0.375207,0.466206,6.477863,0.051827,0


Easy-to-Remember Cluster Performance Table:
Cluster	Current State	Examples & Notes


0	😐 Mixed	JSWSTEEL.NS - Growth, Others Stagnant


1	📈 Strong Growth	BHARTIARTL.NS, NTPC.NS, SBIN.NS


2	📉 Underperforming	ASIANPAINT.NS, HDFCLIFE.NS, KOTAKBANK.NS


3	🌱 Moderate Growth	APOLLOHOSP.NS, DRREDDY.NS, TATACONSUM.NS


4	😟 Challenges & Uncertainty	BAJFINANCE.NS, HDFCBANK.NS, INFY.NS


5	🚀 Outperforming	ADANIPORTS.NS, BPCL.NS, COALINDIA.NS


6	📈🚀 High Growth, High Beta	BAJAJ-AUTO.NS, ONGC.NS


7	👍 Positive Performance	CIPLA.NS, HEROMOTOCO.NS, SUNPHARMA.NS


8	📈🎢 Growth with Volatility	HCLTECH.NS, ICICIBANK.NS, WIPRO.NS


9	🚀🚀 High Growth, High Volatility	ADANIENT.NS - Potential for Big Gains/Losses

Analysis of Current Stock Clusters:
Based on the provided data, here's an interpretation of the current state of each cluster:


Cluster 0:
Mixed Performance: This cluster shows a mix of performance, with some stocks like JSWSTEEL.NS experiencing moderate growth while others like NESTLEIND.NS and TITAN.NS remaining relatively stagnant.
Potential Turnaround: The positive performance of JSWSTEEL.NS might indicate a potential turnaround or positive developments within its sector.


Cluster 1:
Growth & Stability: Stocks in this cluster are displaying strong growth, as seen with BHARTIARTL.NS, NTPC.NS, POWERGRID.NS, and SBIN.NS.
Positive Investor Sentiment: This could signify positive investor sentiment and potential future growth within these companies and their respective sectors.


Cluster 2:
Underperformance: This cluster currently consists of stocks experiencing negative returns, including ASIANPAINT.NS, HDFCLIFE.NS, HINDUNILVR.NS, KOTAKBANK.NS, LTIM.NS, and UPL.NS.
Potential Concerns: This might indicate challenges or negative developments within their sectors or specific company issues impacting investor confidence.


Cluster 3:
Moderate Growth: Stocks in this cluster are exhibiting moderate growth and positive returns, including APOLLOHOSP.NS, DRREDDY.NS, EICHERMOT.NS, GRASIM.NS, and TATACONSUM.NS.
Steady Performance: This suggests stable performance and potential for continued growth within these companies.


Cluster 4:
Challenges and Uncertainty: This cluster is facing challenges, with stocks like BAJAJFINSV.NS, BAJFINANCE.NS, HDFCBANK.NS, INDUSINDBK.NS, INFY.NS, ITC.NS, and ULTRACEMCO.NS experiencing negative returns.
Economic or Sector-Specific Issues: This could be due to broader economic concerns, sector-specific challenges, or company-specific issues impacting investor confidence.


Cluster 5:
Outperformance: This cluster is demonstrating strong performance, with ADANIPORTS.NS, BPCL.NS, and COALINDIA.NS showing significant positive returns.
Positive Developments: This suggests potential positive developments or increased investor interest within these companies and their sectors.


Cluster 6:
Strong Growth & High Beta: BAJAJ-AUTO.NS and ONGC.NS in this cluster are exhibiting high growth, aligning with their high beta characteristics.
Investor Confidence: This indicates strong investor confidence and potential for further growth in these companies.


Cluster 7:
Positive Performance: This cluster is performing well, with CIPLA.NS, HEROMOTOCO.NS, HINDALCO.NS, MARUTI.NS, RELIANCE.NS, and SUNPHARMA.NS showing positive returns.
Growth Potential: This suggests potential for continued growth and positive investor sentiment surrounding these companies.


Cluster 8:
Growth with Some Volatility: DIVISLAB.NS, HCLTECH.NS, ICICIBANK.NS, and WIPRO.NS in this cluster are experiencing growth but with a degree of volatility.
Potential for Fluctuation: This indicates potential for future price fluctuations and the need for careful monitoring of these stocks.


Cluster 9:
High Growth & Volatility: ADANIENT.NS is exhibiting high growth aligning with its high-risk, high-reward characteristics.


Potential for Significant Gains or Losses: Investors should be aware of the potential for significant gains but also substantial losses due to the inherent volatility of this stock.